<a href="https://colab.research.google.com/github/tharushaliyanagama/OralCancerEarlyDetection-DSGP/blob/Image-Validation/Validation_DSGP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import layers, models
import os
import random
import shutil

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Paths
main_dataset_dir = "/content/drive/MyDrive/Train"
output_dir = "/content/drive/MyDrive/Divided_DSGP"
train_split = 0.8         # 80% for training, 20% for validation

# Create train/val folders
os.makedirs(os.path.join(output_dir, "train/mouth"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "train/non_mouth"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "val/mouth"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "val/non_mouth"), exist_ok=True)

In [4]:
# Function to split data
def split_data(class_name):
    class_dir = os.path.join(main_dataset_dir, class_name)
    images = os.listdir(class_dir)
    random.shuffle(images)
    split_index = int(len(images) * train_split)

    train_images = images[:split_index]
    val_images = images[split_index:]

    # Move files to train and val folders
    for img in train_images:
        src = os.path.join(class_dir, img)
        dst = os.path.join(output_dir, "train", class_name, img)
        shutil.copy(src, dst)

    for img in val_images:
        src = os.path.join(class_dir, img)
        dst = os.path.join(output_dir, "val", class_name, img)
        shutil.copy(src, dst)

In [5]:
# Split for "mouth" and "non_mouth"
split_data("mouth")
split_data("non_mouth")

print("Dataset split into training and validation sets!")

Dataset split into training and validation sets!


In [6]:
# Paths to your dataset
data_dir = "/content/drive/MyDrive/Divided_DSGP"
train_dir = os.path.join(data_dir, "train")  # Replace with your dataset's train folder
val_dir = os.path.join(data_dir, "val")      # Replace with your dataset's validation folder

In [7]:
# Parameters
batch_size = 32
img_height = 224
img_width = 224

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

Found 2603 images belonging to 2 classes.
Found 651 images belonging to 2 classes.


In [8]:
# Load a pre-trained model (MobileNetV2)
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

# Build the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification: Mouth / Not Mouth
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

# Save the trained model
model.save("mouth_detection_model.h5")

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


54/82 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.8612 - loss: 0.3575

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


82/82 ━━━━━━━━━━━━━━━━━━━━ 207s 2s/step - accuracy: 0.8737 - loss: 0.3228 - val_accuracy: 0.9478 - val_loss: 0.1411
Epoch 2/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 193s 2s/step - accuracy: 0.9382 - loss: 0.1605 - val_accuracy: 0.9585 - val_loss: 0.1436
Epoch 3/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 322s 4s/step - accuracy: 0.9478 - loss: 0.1403 - val_accuracy: 0.9585 - val_loss: 0.1114
Epoch 4/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 206s 2s/step - accuracy: 0.9524 - loss: 0.1403 - val_accuracy: 0.9601 - val_loss: 0.0996
Epoch 5/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 218s 3s/step - accuracy: 0.9584 - loss: 0.1221 - val_accuracy: 0.9585 - val_loss: 0.1005
Epoch 6/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 260s 3s/step - accuracy: 0.9575 - loss: 0.1157 - val_accuracy: 0.9601 - val_loss: 0.1087
Epoch 7/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 187s 2s/step - accuracy: 0.9638 - loss: 0.0997 - val_accuracy: 0.9677 - val_loss: 0.1044
Epoch 8/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 198s 2s/step - accuracy: 0.9554 - loss: 0.0984 - val_accuracy: 0.9570 - val_loss: 0.

In [10]:
# Function to validate an image
def validate_image(image_path, model_path="mouth_detection_model.h5"):
    model = tf.keras.models.load_model(model_path)
    image = load_img(image_path, target_size=(img_height, img_width))
    image_array = img_to_array(image) / 255.0
    image_array = tf.expand_dims(image_array, axis=0)  # Add batch dimension
    prediction = model.predict(image_array)[0][0]
    if prediction > 0.5:
        return "Inside of a mouth"
    else:
        return "Not inside of a mouth"